In [1]:
import numpy as np
import nilearn
from nilearn import plotting
import pandas as pd
import os
import seaborn as sns
from nilearn.connectome import sym_matrix_to_vec
from modeling_utils import *
import networkx as nx

In [2]:
diagnosis = pd.read_csv("participants.csv")["diagnosis"].values

In [3]:
scz_features_conn = np.load("all_features.npy")[np.where(diagnosis == 1)[0], :499500]
nc_features_conn = np.load("all_features.npy")[np.where(diagnosis == 0)[0], :499500]
feature_importance = np.load("results/importance_conn.npy")


In [74]:
n = 20000

In [75]:
def weighted_degree_centrality(G):
    centrality = {}
    for node in G:
        weighted_degree = sum(weight for _, _, weight in G.edges(node, data='weight'))
        centrality[node] = weighted_degree
    return centrality

In [76]:
avg_conn_matrix_subset_scz = conn_features_to_matrix(100, feature_importance, scz_features_conn)
avg_conn_matrix_subset_nc = conn_features_to_matrix(100, feature_importance, nc_features_conn)

In [77]:
# np.save("mat_subset_scz.npy", avg_conn_matrix_subset_scz)
# np.save("mat_subset_nc.npy", avg_conn_matrix_subset_nc)

In [78]:
G_scz = nx.from_numpy_array(avg_conn_matrix_subset_scz)
G_nc = nx.from_numpy_array(avg_conn_matrix_subset_nc)

In [79]:
# ecentrality_scz = nx.eigenvector_centrality_numpy(G_scz, weight='weight')
# ecentrality_nc = nx.eigenvector_centrality_numpy(G_nc, weight='weight')
# lcentrality_nc = nx.laplacian_centrality(G_nc, normalized=True)
dcentrality_weighted_scz = weighted_degree_centrality(G_scz)
dcentrality_weighted_nc = weighted_degree_centrality(G_nc)


In [80]:
# ecentrality_scz = np.array(list(ecentrality_scz.values()))
# ecentrality_nc = np.array(list(ecentrality_nc.values()))
# lcentrality_nc = np.array(list(lcentrality_nc.values()))

In [81]:
dcentrality_weighted_scz = np.array(list(dcentrality_weighted_scz.values()))
dcentrality_weighted_nc = np.array(list(dcentrality_weighted_nc.values()))

In [82]:
# np.save("results/eigenvector_centrality_scz.npy", ecentrality_scz)
# np.save("results/eigenvector_centrality_nc.npy", ecentrality_nc)
# np.save("results/laplacian_centrality_nc_norm.npy", lcentrality_nc)

In [83]:
np.save(f"results/degree_centrality_nc_{n}.npy", dcentrality_weighted_nc)
np.save(f"results/degree_centrality_scz_{n}.npy", dcentrality_weighted_scz)


In [88]:
dcentrality_weighted_scz.shape

(1000,)

In [73]:
G_imp = nx.from_numpy_array(imp_matrix)
ecentrality_imp = nx.eigenvector_centrality_numpy(G_imp, weight='weight')
ecentrality_imp = np.array(list(ecentrality_imp.values()))
np.save("results/eigenvector_centrality_imp.npy", ecentrality_imp)

## Mann-Whitney Degree Centrality

In [91]:
from scipy.stats import mannwhitneyu
n_features = [100, 500, 1000, 5000, 10000, 20000]

for n in n_features:
    scz_features_conn = np.load("all_features.npy")[np.where(diagnosis == 1)[0], :499500]
    nc_features_conn = np.load("all_features.npy")[np.where(diagnosis == 0)[0], :499500]

    top_indices = np.argsort(-feature_importance)[:n]
    mask = np.ones(scz_features_conn.shape[1], dtype=bool)
    mask[top_indices] = False

    scz_features_conn[:, mask] = np.nan
    nc_features_conn[:, mask] = np.nan
    
    deg_centrality_scz = np.empty((scz_features_conn.shape[0], 1000))
    deg_centrality_nc = np.empty((nc_features_conn.shape[0], 1000))
    
    for i, _ in enumerate(deg_centrality_scz):
        mat_scz = conn_features_to_matrix(n, feature_importance,scz_features_conn[i], mean = False)
        G_scz = nx.from_numpy_array(mat_scz)
        dcentrality_weighted_scz = weighted_degree_centrality(G_scz)
        dcentrality_weighted_scz = np.array(list(dcentrality_weighted_scz.values()))
        deg_centrality_scz[i] = dcentrality_weighted_scz
    np.save(f"results/degree_centrality_scz_{n}.npy", deg_centrality_scz)
        
    for i, _ in enumerate(deg_centrality_nc):
        mat_nc = conn_features_to_matrix(n, feature_importance,nc_features_conn[i], mean = False)
        G_nc = nx.from_numpy_array(mat_nc)
        dcentrality_weighted_nc = weighted_degree_centrality(G_nc)
        dcentrality_weighted_nc = np.array(list(dcentrality_weighted_nc.values()))
        deg_centrality_nc[i] = dcentrality_weighted_nc
    np.save(f"results/degree_centrality_nc_{n}.npy", deg_centrality_nc)
        
    whitneyu_results = np.empty((1000, 2))
    
    for i, _ in enumerate(deg_centrality_scz.T):
        whitneyu_results[i] = mannwhitneyu(deg_centrality_scz[:, i], deg_centrality_nc[:, i])
        
    np.save(f"results/deg_centrality_whitneyu_{n}.npy", whitneyu_results)



In [92]:
deg_centrality_scz.shape

(248, 1000)

In [96]:
np.load(f"results/degree_centrality_scz_5000.npy").shape

(248, 1000)